<a href="https://colab.research.google.com/github/moh2-lh/moh2-lh/blob/main/Copy_of_05_CNN_Architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

© 2021 Zaka AI, Inc. All Rights Reserved

#CNN Architectures
**Objective:** The goal from this exercise is to get familiar with the codes that built famous CNN models like AlexNet and VGG19. It also introduces three types of Transfer Learning Usages with pre-trained classifier, standalone feature extractor and pre-trained model with custom classifier.

## Model Architectures Build
In this section, we will observe how the AlexNet and VGG models were built with code.

### AlexNet

Code to build the AlexNet Architecture, winner of ILSVRC in 2012.

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model

in_layer = layers.Input((227,227,3))
conv1 = layers.Conv2D(96, 11, strides=4, activation='relu')(in_layer)
pool1 = layers.MaxPool2D(3, 2)(conv1)
conv2 = layers.Conv2D(256, 5, strides=1, padding='same', activation='relu')(pool1)
pool2 = layers.MaxPool2D(3, 2)(conv2)
conv3 = layers.Conv2D(384, 3, strides=1, padding='same', activation='relu')(pool2)
conv4 = layers.Conv2D(256, 3, strides=1, padding='same', activation='relu')(conv3)
pool3 = layers.MaxPool2D(3, 2)(conv4)
flattened = layers.Flatten()(pool3)
dense1 = layers.Dense(4096, activation='relu')(flattened)
drop1 = layers.Dropout(0.5)(dense1)
dense2 = layers.Dense(4096, activation='relu')(drop1)
drop2 = layers.Dropout(0.5)(dense2)
preds = layers.Dense(1000, activation='softmax')(drop2)

model = Model(in_layer, preds)
model.compile(loss="categorical_crossentropy", optimizer='sgd', metrics=["accuracy"])

model.summary()

### VGG19
Code to build the VGG16 and VGG19 Architectures, winner of ILSVRC in 2014.

In [ ]:
# Code to build VGG Architecture

from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential

from functools import partial

conv3 = partial(layers.Conv2D,
                kernel_size=3,
                strides=1,
                padding='same',
                activation='relu')

def block(in_tensor, filters, n_convs):
    conv_block = in_tensor
    for _ in range(n_convs):
        conv_block = conv3(filters=filters)(conv_block)
    return conv_block

def _vgg(in_shape=(224,224,3),
         n_classes=1000,
         opt='sgd',
         n_stages_per_blocks=[2, 2, 3, 3, 3]):
    in_layer = layers.Input(in_shape)

    block1 = block(in_layer, 64, n_stages_per_blocks[0])
    pool1 = layers.MaxPool2D()(block1)
    block2 = block(pool1, 128, n_stages_per_blocks[1])
    pool2 = layers.MaxPool2D()(block2)
    block3 = block(pool2, 256, n_stages_per_blocks[2])
    pool3 = layers.MaxPool2D()(block3)
    block4 = block(pool3, 512, n_stages_per_blocks[3])
    pool4 = layers.MaxPool2D()(block4)
    block5 = block(pool4, 512, n_stages_per_blocks[4])
    pool5 = layers.MaxPool2D()(block5)
    flattened = layers.GlobalAvgPool2D()(pool5)
  
    # DNN
    dense1 = layers.Dense(4096, activation='relu')(flattened)
    dense2 = layers.Dense(4096, activation='relu')(dense1)
    preds = layers.Dense(1000, activation='softmax')(dense2)

    model = Model(in_layer, preds)
    model.compile(loss="categorical_crossentropy", optimizer=opt,
	              metrics=["accuracy"])
    return model

def vgg16(in_shape=(224,224,3), n_classes=1000, opt='sgd'):
    return _vgg(in_shape, n_classes, opt)

def vgg19(in_shape=(224,224,3), n_classes=1000, opt='sgd'):
    return _vgg(in_shape, n_classes, opt, [2, 2, 4, 4, 4])

model = vgg19()
model.summary()

###Importing VGG19()
An easier and better way to use VGG model architecture.


In [ ]:
# example of loading the vgg19 model
from tensorflow.keras.applications.vgg19 import VGG19

# load model
model = VGG19()

# summarize the model
model.summary()

##Transfer Learning

### Pretrained classifier

How to use a pretrained classifier to directly predict on images. First, getting an image to work on using `curl`

In [ ]:
# let's download an image from the internet
!curl -o dog.jpg https://images.pexels.com/photos/356378/pexels-photo-356378.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500

Preparing the image and calling the VGG model to predict with it the class of the object in the image

In [ ]:
# example of using a pre-trained model as a classifier
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16

# load an image from file
image = load_img('dog.jpg', target_size=(224, 224))

# convert the image pixels to a numpy array
image = img_to_array(image)

# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

# prepare the image for the VGG model
image = preprocess_input(image)

# load the model
model = VGG16()

# predict the probability across all output classes
yhat = model.predict(image)

# convert the probabilities to class labels
label = decode_predictions(yhat)

print(label)

# retrieve the most likely result, e.g. highest probability
label = label[0][0]

# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

### Standalone Feature Extractor

How to use the pretrained VGG model as a standalone feature extractor and using the same image from before.

In [ ]:
# example of using the vgg16 model as a feature extraction model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

# load an image from file
image = load_img('dog.jpg', target_size=(224, 224))

# convert the image pixels to a numpy array
image = img_to_array(image)

# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

# prepare the image for the VGG model
image = preprocess_input(image)

# load model
model = VGG16()

# remove the output layer
my_model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

print("Passing image to model ...")

# get extracted features
features = my_model.predict(image)
print("Output shape: {}".format(features.shape))

### Integrated Feature Extractor
We can also use the model architecture for feature extraction and integrate it into a new classification model. This means we load the CNN architecture without the fully connected layers and then build these ourselves.

In [ ]:
# example of tending the vgg16 model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

# load model without classifier layers
model = Sequential()
model.add(VGG16(include_top=False, input_shape=(300, 300, 3)))

for layer in model.layers:
  layer.trainable = False

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))

# summarize
model.summary()